## Pobieranie Pakietów

In [3]:
!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q


Pakiet accelerate jest wymagany przez inne pakiety do efektywnego uruchamiania modeli na wielu kartach graficznych/TPU.
<br><br>
bitsandbytes pozwoli nam na załadowanie modelu w trybie ograniczonej precyzji wag. Prowadzi to do istotnego zmniejszenia wymagań pamięciowych modelu.

## Ładowanie Bielika

In [4]:
import pandas as pd  # Working with dataset
import torch  # Deep learning framework

from torch.utils.data import DataLoader, Dataset
from huggingface_hub import hf_hub_download  # Downloading Poquad dataset from hugginface manually.

import json  # Saving new dataset
import warnings

from math import ceil

warnings.filterwarnings("ignore")

Poniższa komórka:
- pobiera model bielik, ładuje do pamięci wagi modelu oraz tokenizera
- Tworzy ustawienia generowania (tj temperature, max_tokens, top_k, top_p).
- Ustawia niższą precyzję reprezentowania wag (kwantyzacja) za pomocą BitsAndBytes.
-A także sprawia, że tekst generowany przez model będzie wyświetlał się w czasie rzeczywistym (TextStreamer).

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer, Trainer, Seq2SeqTrainer, Seq2SeqTrainingArguments

device = "cuda"
model_name = 'speakleash/Bielik-7B-Instruct-v0.1'


temperature = 1.0
max_tokens = 500
top_k = 200
top_p = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.bfloat16,
                                            quantization_config=quantization_config
                                            )

model.generation_config.pad_token_id = tokenizer.pad_token_id


tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Poniższa komórka pozwala nam uprościć generowanie tekstu przez model do postaci pojedynczej funkcji generate, przyjmującej treść prompta oraz treść systemowego prompta. <br><br> Dzięki TextStreamerowi zdefiniowanemu wcześniej funkcja będzie wyświetlała generowany tekst w czasie rzeczywistym. Po wygenerowaniu całości funkcja zwraca tekst w postaci stringu (typ danych zmiennych tekstowych).

In [5]:
def generate(prompt, system = None):

  messages = []

  if system:
    messages.append( {"role": "system", "content": system })

  messages.append( {"role": "user", "content": prompt })


  tokenizer_output = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)

  if torch.cuda.is_available():
    model_input_ids = tokenizer_output.input_ids.to(device)

    model_attention_mask = tokenizer_output.attention_mask.to(device)

  else:
    model_input_ids = tokenizer_output.input_ids
    model_attention_mask = tokenizer_output.attention_mask


  # generowanie odpowiedzi
  outputs = model.generate(model_input_ids,
                           attention_mask=model_attention_mask,
                           streamer = streamer,
                           max_new_tokens=max_tokens,
                           do_sample=True if temperature else False,
                           temperature = temperature,
                           top_k = top_k,
                           top_p = top_p)

  return tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [6]:
# Testowe działanie generowania.
query = "Z Ilu kości składa się czaszka?"
prompt = f"Odpowiedz zwięźle na pytanie: '{query}'"
output = generate(prompt)

 Czaszka składa się z 7 kości.


## Ładowanie Danych Q&A (Poquad) oraz tworzenie funkcji pomocnicznych

"PoQuAD - The Polish Question Answering Dataset - Description and Analysis "
<br>Ryszard Tuora, Aleksandra Zwierzchowska, Natalia Zawadzka-Paluektau, Cezary Klamra, Łukasz Kobyliński
<br> <br>
Ze strony datasetu:
"Systemy QA (Question Answering) są ważnym zastosowaniem technik przetwarzania języka naturalnego. Pozwalają na wyszukiwanie informacji w dużych zbiorach niestrukturyzowanych tekstów w najbardziej naturalny sposób, bez konieczności formułowania zapytania wyszukiwania. Współczesny standard w Question Answering polega na dostosowywaniu ogólnych modeli językowych do zadaniowych zbiorów danych. Zadanie może być sformułowane na różne sposoby, w zależności od wykorzystania zasobów tekstowych i formy ostatecznej odpowiedzi. Jednym z popularniejszych formatów jest tzw. Machine Reading Comprehension (MRC 33), gdzie odpowiedź na pytanie musi być oparta na zrozumieniu dostarczonego fragmentu tekstu. PoQuAd jest pierwszym zbiorem danych MRC w języku polskim. Dane zostały zebrane z Wikipedii, a dzięki procesowi starannej anotacji udało się uzyskać ponad 70 tysięcy punktów danych."


Poquad jest wymagającym datasetem testujący umiejętności modelu w znajdowaniu informacji w tekście, a także w krytycznym czytaniu tekstów ze zrozumieniem. Główna trudność tych danych polega na istnieniu przykładów pytań, na które nie da się odpowiedzieć na podstawie tekstu. W takich przypadkach model powinien wstrzymywać się przed udzielaniem odpowiedzi. Wymaga do głębokiego zrozumienia tekstu oraz umiejętności analizy informacji znajdujących się w tekście. Bielik tak samo jak większość innych modeli o podobnym rozmiarze, ma wielkie trudności we wstrzymaniu się przed generowaniem odpowiedzi.

---

Poniższe 3 komórki pobierają dane z serwisu hugginface. W sposób nieautomatyczny ładują je do postaci tabeli danych, a na końcu tworzą nową kolumnę w danych odpowiadającą odpowiedzi, którą oczekujemy od modelu.

In [7]:
hf_hub_download(repo_id="clarin-pl/poquad", filename="poquad-train.json", repo_type="dataset", local_dir="./data")

poquad-train.json:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

'data/poquad-train.json'

In [8]:
def read_poquad_data(filepath):
        whole_data = []
        with open(filepath, encoding="utf-8") as f:
            squad = json.load(f)
            id_ = 0
            for example in squad["data"]:
                title = example.get("title", "")
                for paragraph in example["paragraphs"]:
                    context = paragraph["context"]
                    for qa in paragraph["qas"]:
                        question = qa["question"]

                        answer_starts = [answer["answer_start"] for answer in qa["answers"]] if "answers" in qa else []

                        answers = [answer["text"] for answer in qa["answers"]] if "answers" in qa else []
                        is_impossible = qa["is_impossible"]

                        id_ += 1
                        whole_data.append({
                            "id": id_,
                            "title": title,
                            "context": context,
                            "question": question,
                            "is_impossible" : is_impossible,
                            "answers": {
                                "answer_start": answer_starts,
                                "text": answers,
                            },
                        })

        return pd.DataFrame(whole_data).set_index("id")

In [9]:
df = read_poquad_data("./data/poquad-train.json")

df["target"] = df["answers"].apply(lambda x: "Odpowiedź: " + x["text"][0] if x["text"] else "Odpowiedź niemożliwa!")

Poniższy kod tworzy funkcje, które ułatwią nam (oraz modelowi) interakcję z danymi. Pierwsza z nich to print_poquad_row(), która pozwala nam na spojrzenie na dowolny wiersz w danych w sposób czytelny i uporządkowany. Druga z nich to create_prompt_from_row(), która pozwala nam na automatyczne stworzenie promptu dla bielika, na podstawie numeru wiersza z danych.

W tej komórce znajduje się również few_shot_text, czyli zmienna przechowująca przydatne przykłady, z których bielik może nauczyć się udzielać poprawnie sformatowanej odpowiedzi (in-context learning). <br> Podczas konstruowania tych demonstracji, zaproponowałem metody wnioskowania w oparciu o 4 wiersze z datasetu. Te 4 przykłady wybrałem samodzielnie z danych, tak aby miały możliwie jak najkrótsze konteksty. W ten sposób mogłem zmieścić 4 pełne przykłady jednocześnie utrzymując rozmiar prompta na poziomie, który bezproblemowo mieści się w modelu.

In [10]:
def print_using_shorter_lines(words_list, word_limit):
  n_fragments = ceil(len(words_list)/word_limit)

  for i in range(n_fragments):
    print(" ".join(words_list[12*i:12*(i+1)]))


def print_poquad_row(df, i, n_words_in_line=12):
  print("Kontekst:")

  context = df.loc[i, "context"].split(" ")
  print_using_shorter_lines(context, n_words_in_line)

  print("\nPytanie:")
  question = df.loc[i, "question"].split(" ")
  print_using_shorter_lines(question, n_words_in_line)

  print("\nOczekiwana odpowiedź:")
  answer = df.loc[i, "target"].split(" ")
  print_using_shorter_lines(answer, n_words_in_line)


few_shot_text = f"""
# PRZYKŁADOWE FORMY ODPOWIEDZI:
Przykład 1)
Kontekst: {df.loc[26070, "context"]}
Pytanie: {df.loc[26070, "question"]}
{df.loc[26070, "target"]}

Wnioskowanie: Wyrażenie 'związane jest' wskazuje na wpływ przecięcia się szlaków kupiekcich z powstaniem Lubina.

Przykład 2)
Kontekst: {df.loc[25251, "context"]}
Pytanie: {df.loc[25251, "question"]}
{df.loc[25251, "target"]}

Wnioskowanie: W tekście nie podano liczby głosów w wyborach na prezydenta stolicy, a jedynie informacje o ilości głosów w wyborach parlamentarnych. Pytano o wybory na prezydenta stolicy Polski, a więc brak informacji w tekście.

Przykład 3)
Kontekst: {df.loc[7900, "context"]}
Pytanie: {df.loc[7900, "question"]}
{df.loc[7900, "target"]}

Wnioskowanie: W tekście informacja o liczbie posłów pojawia się w zdaniu: 'Do Sejmu Ustawodawczego weszło razem 36 posłów PPSD i PPS'. W tekście podano liczbę posłów PPSD i PPS razem wziętych. Nie da się podać tylko liczby posłów PPSD.

Przykład 4)
Kontekst: {df.loc[42005, "context"]}
Pytanie: {df.loc[42005, "question"]}
{df.loc[42005, "target"]}

Wnioskowanie: Z tekstu wynika że Dąbrowski przejął dowodzenie po Sucharskim. W tym samym zdaniu Sucharskiemu przypisywane jest załamanie nerwowe. 'W wyniku' sugeruje wpływ załamania na zmianę dowodzenia. A więc odpowiedź to 'załamania nerwowego'/

# KONIEC PRZYKŁADÓW.
"""


def create_prompt_from_row(df, i, answer=True):
  task_clarification = "udowodnij, że odpowiedź nie pojawia się w tekście, a także zwróć szczególną uwagę na możliwość istnienia dystraktorów odwracających uwagę od poprawnej odpowiedzi." if df.loc[i, "target"] == "Odpowiedź niemożliwa!" else "we wnioskowaniu wyjaśnij dlaczego możemy mieć pewność, że odpowiedź prawidłowo odpowiada na zadane pytanie i znajduje się w tekście."

  return (f"Utwórz wnioskowanie argumentujące na rzecz podanej odpowiedzi, {task_clarification} Nie powtarzaj ani nie streszczaj treści kontekstu. Generuj wyłącznie oryginalny tekst. Pamiętaj, że każdy przykład wymaga innej metody argumentacji, a więc zadanie, które musisz rozwiązać będzie musiało wykorzystywać metody inne niż podane w przykładach."
      + (few_shot_text if answer else "")
      + "\n# Dane dla ciebie"
      + "\nKontekst: "
      + df.loc[i, "context"]
      + "\nPytanie: "
      + df.loc[i, "question"]
      + ("\n" if answer else "")
      + (df.loc[i, "target"] + "\n" if answer else ""))




### Demonstracja funkcji:

In [11]:
print_poquad_row(df, 1)

Kontekst:
Projekty konfederacji zaczęły się załamywać 5 sierpnia 1942. Ponownie wróciła kwestia monachijska,
co uaktywniło się wymianą listów Ripka – Stroński. Natomiast 17 sierpnia 1942
doszło do spotkania E. Beneša i J. Masaryka z jednej a Wł.
Sikorskiego i E. Raczyńskiego z drugiej strony. Polscy dyplomaci zaproponowali podpisanie układu
konfederacyjnego. W następnym miesiącu, tj. 24 września, strona polska przesłała na ręce
J. Masaryka projekt deklaracji o przyszłej konfederacji obu państw. Strona czechosłowacka projekt
przyjęła, lecz już w listopadzie 1942 E. Beneš podważył ideę konfederacji. W
zamian zaproponowano zawarcie układu sojuszniczego z Polską na 20 lat (formalnie nastąpiło
to 20 listopada 1942).

Pytanie:
Co było powodem powrócenia konceptu porozumieniu monachijskiego?

Oczekiwana odpowiedź:
Odpowiedź: wymianą listów Ripka – Stroński


In [12]:
print(create_prompt_from_row(df, 1))

Utwórz wnioskowanie argumentujące na rzecz podanej odpowiedzi, we wnioskowaniu wyjaśnij dlaczego możemy mieć pewność, że odpowiedź prawidłowo odpowiada na zadane pytanie i znajduje się w tekście. Nie powtarzaj ani nie streszczaj treści kontekstu. Generuj wyłącznie oryginalny tekst. Pamiętaj, że każdy przykład wymaga innej metody argumentacji, a więc zadanie, które musisz rozwiązać będzie musiało wykorzystywać metody inne niż podane w przykładach.
# PRZYKŁADOWE FORMY ODPOWIEDZI:
Przykład 1)
Kontekst: Powstanie pierwszej osady na terenie dzisiejszego Lubina (osiedle Stary Lubin) związane jest z przecięciem się w tym miejscu ważnych szlaków kupieckich (np. z Magdeburga poprzez Wrocław do Kijowa i z południa przez Wielkopolskę nad Morze Bałtyckie) w związku z czym była zapewne miejscem odpoczynku wędrujących kupców, a także handlu ich towarami. Nie do końca wiadomo, czy Lubinem władali wówczas Dziadoszanie czy Trzebowianie, gdyż leżał on w połowie drogi między ich głównymi grodami.
Pytanie

---

## Jak bielik wykonuje zadanie w sposób domyślny?

Poniższe komórki prezentują jak Bielik radzi sobie z postawionym zadaniem bez finetuningu oraz z minimalnym wsparciem ze strony prompt engineeringu. Mają one na celu pokazać ograniczenia modeli językowych w sytuacjach gdy zadanie wymaga głębokiego zrozumienia tekstu.

In [13]:
temperature = 0
max_tokens = 150
top_k = 200
top_p = 1

In [14]:
prompt_possible = create_prompt_from_row(df, 1, answer=False)

In [15]:
system_prompt = """Jesteś chatbotem odpowiadającym zwięźle na pytania na podstawie podanego kontekstu. Niektóre pytania mogą być niemożliwe, w takim wypadku musisz wstrzymać się od odpowiedzi. Twoje odpowiedzi mają przyjąć formę:
"Odpowiedź: 'treść odpowiedzi'"
jeśli treść odpowiedź istnieje w tekście
lub
"Odpowiedź niemożliwa!"
jeśli nie da się odpowiedzieć na pytanie na podstawie kontekstu."""

output = generate(prompt=prompt_possible, system=system_prompt)

 Odpowiedź: W sierpniu 1942, kwestia monachijska uaktywniła się poprzez wymianę listów Ripka - Stroński. To wznowienie dyskusji na temat porozumienia monachijskiego było prawdopodobnie spowodowane napięciami politycznymi i strategicznymi w regionie, a także potrzebą ustalenia wspólnych celów i zasad w obliczu rosnącego zagrożenia ze strony III Rzeszy.


**Model Generuje przesadnie rozległy tekst.**




---

In [16]:
print_poquad_row(df, 8) # Row showcase

Kontekst:
Samoloty Pomilio PD weszły na wyposażenie jednostek bojowych i zarazem do walki
od lipca 1917 roku. Pierwsze otrzymały je w tym miesiącu eskadry (Squadriglia)
nr 131 i 132, gdzie zastąpiły Pomilio PC. Do działań wchodziły następnie
kolejne nowo formowane eskadry Pomilio: 133 (we wrześniu), 134 (w październiku), 135
(w styczniu 1918 roku), 136 (w maju 1918 roku). Eskadra 139, sformowana
z dwóch sekcji początkowo przydzielonych do lotnictwa Marynarki (od sierpnia 1917 roku),
weszła do działań w styczniu 1918 roku, a w czerwcu odznaczyła się
w bitwie nad Piawą, zrzucając 175 bomb. W bitwie tej brały udział
też inne eskadry. Model PD został szybko zastąpiony w jednostkach przez PE
i skierowany głównie do szkolenia.

Pytanie:
Ile bomb zrzucono na eskadrę 139?

Oczekiwana odpowiedź:
Odpowiedź niemożliwa!


In [17]:
prompt_impossible = create_prompt_from_row(df, 8, answer=False)

In [18]:
output = generate(prompt=prompt_impossible, system=system_prompt) # Should be "Odpowiedź niemożliwa!"

 Odpowiedź: 175 bomb

Wyjaśnienie: W kontekście podanym, informacja o 175 bombach zrzuconych na eskadrę 139 przez tę jednostkę jest bezpośrednio wymieniona, co oznacza, że ta liczba bomb istotnie była zrzucona podczas bitwy nad Piawą w czerwcu 1918 roku. Dane te nie są wymienione wprost w odpowiedzi na pytanie o zestrzelenia i straty materialne w bitw


**Model Halucynuje.**

---

## Generowanie przesłanek


Zadanie nie jest jednak poza zasięgiem AI. Dzięki wykorzystaniu zdolności modeli do generowania danych ulepszonych o dodatkowe informacje, istnieje możliwości wygenerowania datasetu wyższej jakości niż ten oryginalny oraz wytrenowanie/fine-tuning kolejnego modelu na ulepszonej wersji datasetu, co w rezultacie może istotnie poszerzyć możliwości modelu. <br><br>
W praktyce na podstawie oryginalnych danych, poprosimy model o wygenerowanie procesu wnioskowania, które może pomóc w odnalezieniu poprawnej odpowiedzi (albo wstrzymanie się od odpowiedzi, kiedy ta nie istnieje). Potem te wnioski można dodać do oryginalnych danych, uzyskując poszerzony dataset. I w ostatnim kroku dotrenować model, tak aby na podstawie kontekstu nie tylko odpowiadał na pytania, ale też posiłkował się samodzielnie wygenerowanym procesem wnioskowania potencjalnie zwiększając swoje możliwości.

**Uwaga! ze względu na wysokie wymagania obliczeniowe procesu data generation oraz finetuningu kolejne komórki będą jedynie demonstrowały całą koncepcję na przykładzie kilku obserwacji, stworzenie nowego modelu wymagałoby tysięcy danych powiększonych o procesy wnioskowania. A to wymagałoby godzin generowania tekstu.**

In [19]:
system_prompt = f"""Jesteś AI wyjaśniającym dlaczego podane odpowiedzi odpowiadają na zadane pytanie do kontekstu. Czasem odpowiedzi są niemożliwe, w takim wypadku musisz wyjaśnić dlaczego odpowiedź nie znajduje się w tekście.
Kiedy w tekście brakuje odpowiedzi na pytanie postaraj się wskazać dystraktory znajdujące się w tekście, które mogłyby mylnie wskazywać na bycie odpowiedzią. Np kiedy pytanie odnosi się do pewnego miejsca lub osoby, a tych miejsc i osób nie ma w tekście, to należy wskazać jakie miejsca lub osoby pojawiające się w tekście mogłyby udawać odpowiedź ale nią nie są.
Zachowaj zwięzłą formę wypowiedzi, nie powtarzaj treści kontekstu, twoja odpowiedź musi składać się jedynie z procesu wnioskowania.
Swoją wypowiedź musisz rozpocząć od wyrażenia 'Wnioskowanie: ', do którego w następnym kroku dołączasz swoją argumentację.
Po wypełnieniu pola 'Wnioskowanie' należy zakończyć generowanie odpowiedzi. W szczególności nie możesz od nowa przytaczać całości ani fragmentów tekstu ani też pokazanych ci przykładów.
"""

# generate(prompt=prompt_possible, system=system_prompt)

In [20]:
print(create_prompt_from_row(df, 2))

Utwórz wnioskowanie argumentujące na rzecz podanej odpowiedzi, we wnioskowaniu wyjaśnij dlaczego możemy mieć pewność, że odpowiedź prawidłowo odpowiada na zadane pytanie i znajduje się w tekście. Nie powtarzaj ani nie streszczaj treści kontekstu. Generuj wyłącznie oryginalny tekst. Pamiętaj, że każdy przykład wymaga innej metody argumentacji, a więc zadanie, które musisz rozwiązać będzie musiało wykorzystywać metody inne niż podane w przykładach.
# PRZYKŁADOWE FORMY ODPOWIEDZI:
Przykład 1)
Kontekst: Powstanie pierwszej osady na terenie dzisiejszego Lubina (osiedle Stary Lubin) związane jest z przecięciem się w tym miejscu ważnych szlaków kupieckich (np. z Magdeburga poprzez Wrocław do Kijowa i z południa przez Wielkopolskę nad Morze Bałtyckie) w związku z czym była zapewne miejscem odpoczynku wędrujących kupców, a także handlu ich towarami. Nie do końca wiadomo, czy Lubinem władali wówczas Dziadoszanie czy Trzebowianie, gdyż leżał on w połowie drogi między ich głównymi grodami.
Pytanie

In [21]:
output = generate(prompt=create_prompt_from_row(df, 2), system=system_prompt)

 Wnioskowanie: W sierpniu 1942 odbyło się spotkanie E. Beneša i J. Masaryka z jednej strony, a Wł. Sikorskiego i E. Raczyńskiego z drugiej strony. Z kontekstu wynika, że obie strony uczestniczyły w spotkaniu, dlatego odpowiedź to 'E. Beneša i J. Masaryka z jednej a Wł. Sikorskiego i E. Raczyńskiego'.


In [22]:
output # Raw output, contains system prompt, prompt, and generated rationale

["[INST] <<SYS>>\nJesteś AI wyjaśniającym dlaczego podane odpowiedzi odpowiadają na zadane pytanie do kontekstu. Czasem odpowiedzi są niemożliwe, w takim wypadku musisz wyjaśnić dlaczego odpowiedź nie znajduje się w tekście.\nKiedy w tekście brakuje odpowiedzi na pytanie postaraj się wskazać dystraktory znajdujące się w tekście, które mogłyby mylnie wskazywać na bycie odpowiedzią. Np kiedy pytanie odnosi się do pewnego miejsca lub osoby, a tych miejsc i osób nie ma w tekście, to należy wskazać jakie miejsca lub osoby pojawiające się w tekście mogłyby udawać odpowiedź ale nią nie są.\nZachowaj zwięzłą formę wypowiedzi, nie powtarzaj treści kontekstu, twoja odpowiedź musi składać się jedynie z procesu wnioskowania.\nSwoją wypowiedź musisz rozpocząć od wyrażenia 'Wnioskowanie: ', do którego w następnym kroku dołączasz swoją argumentację.\nPo wypełnieniu pola 'Wnioskowanie' należy zakończyć generowanie odpowiedzi. W szczególności nie możesz od nowa przytaczać całości ani fragmentów tekstu 

In [23]:
output[0].split("[/INST]")[-1].strip().replace("Wnioskowanie: ", "")  # Keep only rationale.

"W sierpniu 1942 odbyło się spotkanie E. Beneša i J. Masaryka z jednej strony, a Wł. Sikorskiego i E. Raczyńskiego z drugiej strony. Z kontekstu wynika, że obie strony uczestniczyły w spotkaniu, dlatego odpowiedź to 'E. Beneša i J. Masaryka z jednej a Wł. Sikorskiego i E. Raczyńskiego'."

Poniższy kod wybiera losowo po 5 obswerwacji dla przypadków gdzie odpowiedź istnieje w tekście oraz gdy odpowiedź nie istnieje. Model generuje wnioskowanie dla każdego z tych przykładów, a potem nowy dataset zapisywany jest do pliku "Poquad_extended_by_rationale.json".
<br><br>
Aby zajrzeć do wnętrza pliku należy otworzyć zakładkę "Pliki" znajdującą się po lewej stronie Colabowego interfejsu, a następnie dwukrotnie kliknąć w nowopowstały plik.

In [24]:
possible_sample = df.loc[~df["is_impossible"] & ~df.index.isin([26070, 25251, 7900, 42005])].sample(5)
impossible_sample = df.loc[df["is_impossible"] & ~df.index.isin([26070, 25251, 7900, 42005])].sample(5)

new_data_contents = {}

for i in possible_sample.index:
  prompt = create_prompt_from_row(possible_sample, i)
  output = generate(prompt=create_prompt_from_row(df, i), system=system_prompt)
  generated_rationale = output[0].split("[/INST]")[-1].strip().replace("Wnioskowanie: ", "")

  data_row_dict = possible_sample.loc[i].to_dict()  # Original data row
  data_row_dict["rationale"] = generated_rationale  # Add rationale to this data.

  new_data_contents[i] = data_row_dict  # Add to dictionary of all rows


for i in impossible_sample.index:
  prompt = create_prompt_from_row(impossible_sample, i)
  output = generate(prompt=create_prompt_from_row(df, i), system=system_prompt)
  generated_rationale = output[0].split("[/INST]")[-1].strip().replace("Wnioskowanie: ", "")

  data_row_dict = impossible_sample.loc[i].to_dict()
  data_row_dict["rationale"] = generated_rationale

  new_data_contents[i] = data_row_dict

# Saves the data with rationales included into a new json file.
with open('Poquad_extended_by_rationale.json', 'w') as fp:
    json.dump(new_data_contents, fp, ensure_ascii=False)



 Wnioskowanie: W pytaniu proszono o historię opowiedzianą w filmie Plac Zbawiciela. We wnioskowaniu należy zidentyfikować odpowiedni fragment w kontekście, który opisuje historię filmu. W tym przypadku, odpowiedni fragment to: "losy porzuconej przez męża kobiety z Warszawy, która próbowała odebrać życie sobie oraz dwóm kilkuletnim synom".

Kontekst: Kolejny film Krauzego, zreal
 Wnioskowanie: W kontekście, pytanie o pseudonim matki Edith Piaf. W odpowiedzi, podajemy pseudonim artystyczny Annetta Maillard. Z tekstu: 'matką – Annetta Maillard (ur. 4 sierpnia 1895 w Livorno, zm. 6 lutego 1945), śpiewaczka znana pod pseudonimem artystycznym „Line Marsa”'. Więc odpowiedź to 'Line Marsa'.
 Wnioskowanie: Zdanie 'Obłońscy toną w długach' wskazuje na przyczynę, która skłania Stiwa do poszukiwania lepiej płatnej posady państwowej. Sytuacja finansowa Obłońskich jest źródłem motywacji Stiwy do zmiany pracy.
 Wnioskowanie: Zdanie 'Polski Związek Łyżwiarstwa Figurowego nie zrezygnował ze wspierania 

Podsumowując, model stworzył procesy wnioskowania, które mogą stać się częścią treningu innego modelu. W następnym kroku chcielibyśmy, aby kolejny model został dotrenowany na danych, w których inputem byłby: <br>
- Kontekst - paragraf z wikipedii
- Pytanie - pytanie do kontekstu<br><br>A oczekiwanym outputem byłaby para:
- Wnioskowanie - wygenerowane w poprzednim kroku
- Odpowiedź

Ale Finetuning to długa historia na oddzielny notatnik...


*PS: Jak można zaobswerwować, wiele brakuje w tym 'wnioskach', można też dopatrzeć się wiele anomalii w wygenerowanym tekście oraz pewnych błędów.<br> Chcąc osiągnąć lepsze efekty można by było:*
- *wprowadzić dalsze usprawnienia w promptach dla modelu*
- *dobrać lepsze parametry generowania tekstu*
- *wygenerowane wnioski ewaluować w innym prompcie, próbować je udoskonalić kolejnymi promptami*
- *I wiele wiele innych metod, na które obecnie nie wpadłem lub jeszcze zostaną odkryte*
